# Baseline
# Out of the Box small Flan T5 model
Results for Test and Validation

In [1]:
!pip install -q -U transformers
!pip install -q -U datasets
!pip install -q -U evaluate
!pip install -q -U tokenizers
!pip install -q -U bitsandbytes
!pip install -q rouge_score

In [2]:
import re
import random
import numpy as np
from scipy.special import softmax
import pprint

import bitsandbytes as bnb

import torch
import transformers
import evaluate
from datasets import Dataset, load_dataset, DatasetDict

# For from-scratch T5 model
from transformers import T5TokenizerFast, T5Config, T5ForConditionalGeneration

# For pre-trained T5 model
from transformers import T5Tokenizer, T5ForConditionalGeneration  # this won't import twice, just noting here what's for each model

# For all T5 models
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

# For BLEURT (to load a trained model for evaluation)
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM


# For style classifier model (also for evaluating the seq2seq model output)
from transformers import BertTokenizer, BertForSequenceClassification, BitsAndBytesConfig,pipeline
from transformers import TrainingArguments, Trainer

import pandas as pd
from tqdm import tqdm

In [3]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Path to data save in Drive
train = 'FairytaleQA_train.csv'
valid = 'FairytaleQA_valid.csv'
test = 'FairytaleQA_test.csv'
path = '/content/drive/MyDrive/266/FinalProject/data/' #DANIELLE personal


In [5]:
train = path+train
valid = path+valid
test = path+test

train = pd.read_csv(train)
valid = pd.read_csv(valid)
test = pd.read_csv(test)

In [6]:
train_ds = Dataset.from_pandas(train, split="train")
test_ds = Dataset.from_pandas(test, split="test")
valid_ds = Dataset.from_pandas(valid, split="test")

# Combine into a single DatasetDict
ds = DatasetDict({
    "train": train_ds,
    "test": test_ds,
    "validation": valid_ds,
})

train = ds['train'].shuffle()
val = ds['validation'].shuffle()
test = ds['test'].shuffle()

In [7]:
test

Dataset({
    features: ['story_name', 'story_section', 'question', 'answer1', 'answer2', 'local_or_sum', 'attribute', 'ex_or_im', 'ex_or_im2'],
    num_rows: 1007
})

In [8]:
"""
Initialize the pipeline with bitsandbytes quantization
"""
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Initialize pipeline
model_id = "google/flan-t5-small"

pipe = pipeline(
   "text2text-generation",
   model=model_id,
   model_kwargs={"torch_dtype": torch.bfloat16, "quantization_config": quantization_config},
   device_map="auto",
   trust_remote_code=True,
   truncation=True
)



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [9]:
# load rouge

rouge = evaluate.load('rouge')

# Load the BLEU metric and the trained BLEURT model for semantic similarity scoring

bleu = evaluate.load("bleu")

bleurt_tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-512")
bleurt_model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-512")

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [33]:
%%time

# Store results for aggregate scoring

def process_dataset(dataset):
  results = []
  bleurt_scores = []
  for idx, sample in enumerate(tqdm(dataset)):

      # if idx >= 10:  # Stop after processing 10 samples
      #   return pd.DataFrame(results)
      #   break

      prefix = "Please answer this question: "
      context = " Context: "

      question = prefix + sample['question'] + context + sample['story_section']

      # Generate summary via the pipeline
      outputs = pipe(
                          question,
                          max_new_tokens=500,
      )

      answer = outputs[0]["generated_text"]

      if answer.isspace():
        print("\nANSWER IS SPACE for question:",sample['question'],"\nReference:", sample['answer1'])
        print("\nanswer:",[answer])
        answer = 'blank'

      # print("\nbegin answer:", answer, ":end answer")
      # print("\n",[answer],"****")

      # Evaluation
      predictions = [answer]
      reference1 = [sample['answer1']]
      reference2 = [sample['answer2']]

      # Calculate ROUGE scores
      rouge_scores1 = rouge.compute(predictions=predictions, references=reference1)
      if reference1 != reference2:
        rouge_scores2 = rouge.compute(predictions=predictions, references=reference2)
      else:
        rouge_scores2 = rouge_scores1
      rouge_scores1 = {k+'_a1': v for k, v in rouge_scores1.items()}
      rouge_scores2 = {k+'_a2': v for k, v in rouge_scores2.items()}


      # Calculate BLEU scores
      bleu_scores1 = bleu.compute(predictions=predictions, references=reference1,smooth=True)
      if reference1 != reference2:
        bleu_scores2 = bleu.compute(predictions=predictions, references=reference2,smooth=True)
      else:
        bleu_scores2 = bleu_scores1
      bleu_scores1 = {k+'_a1': v for k, v in bleu_scores1.items()}
      bleu_scores2 = {k+'_a2': v for k, v in bleu_scores2.items()}

      # Calculate BLEURT scores
      with torch.no_grad():
        bleurt_score_a1 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer1'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        if reference1 != reference2:
          bleurt_score_a2 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer2'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        else:
          bleurt_score_a2 = bleurt_score_a1


      # Store results
      results.append({
          'id': idx,
          'question': sample['question'],
          'ex_or_im': sample['ex_or_im'],
          'story_section': sample['story_section'][:500],  # Store truncated text for readability
          'reference_answer1': sample['answer1'],
          'reference_answer2': sample['answer2'],
          'generated_answer': answer,
          **rouge_scores1,
          **rouge_scores2,
          **bleu_scores1,
          **bleu_scores2,
          'bleurt_score_a1': bleurt_score_a1,
          'bleurt_score_a2': bleurt_score_a2,
      })

      # Print progress update every 10 samples
      if (idx + 1) % 100 == 0:
          print(f"\nProcessed {idx + 1} samples; Results length: {len(results)}")
          print(f"Latest Answer1 ROUGE-1: {rouge_scores1['rouge1_a1']:.4f}")

  return pd.DataFrame(results)


CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11 µs


In [26]:
%%time
test_results_df = process_dataset(test)

 10%|▉         | 100/1007 [05:37<32:55,  2.18s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.0000


 20%|█▉        | 200/1007 [10:42<28:58,  2.15s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.8750


 30%|██▉       | 300/1007 [15:27<40:46,  3.46s/it]

...


Processed 900 samples; Results length: 900
Latest Answer1 ROUGE-1: 0.0000


 99%|█████████▉| 1000/1007 [46:56<00:18,  2.69s/it]


Processed 1000 samples; Results length: 1000
Latest Answer1 ROUGE-1: 0.6667


100%|██████████| 1007/1007 [47:14<00:00,  2.81s/it]

CPU times: user 46min, sys: 12.5 s, total: 46min 13s
Wall time: 47min 14s


In [27]:
print("length of results", len(test_results_df))
test_results_df[:2]

length of results 1007


,id,question,ex_or_im,story_section,reference_answer1,reference_answer2,generated_answer,rouge1_a1,rouge2_a1,rougeL_a1,...,translation_length_a1,reference_length_a1,bleu_a2,precisions_a2,brevity_penalty_a2,length_ratio_a2,translation_length_a2,reference_length_a2,bleurt_score_a1,bleurt_score_a2
0,0,how did the brother feel because they had been...,explicit,"but his brother complained of being weary , an...",weary .,weary .,they were weary,0.5,0.0,0.5,...,3,2,0.0,"[0.3333333333333333, 0.0, 0.0, 0.0]",1.000000,1.5,3,2,-0.087234594,-0.087234594
1,1,how did ian feel after he reached the place wh...,implicit,' bear me to the kingdom of the king under the...,nervous .,excited .,glad,0.0,0.0,0.0,...,1,2,0.0,"[0.0, 0.0, 0.0, 0.0]",0.367879,0.5,1,2,-0.9837408,0.13179041


In [29]:
# save results

results_path = 'drive/MyDrive/266/FinalProject/results/'  #DANIELLE

test_results_df.to_csv(results_path+f'T5baseline_test.csv', index=False)

In [34]:
vx= valid[645:650]
vy = Dataset.from_pandas(vx, split="test")

In [35]:
vz = process_dataset(vy)

  0%|          | 0/10 [00:00<?, ?it/s]


ANSWER IS SPACE for question: who was a mighty hero and a good archer ? 
Reference: hou i .

answer: ['                                                                                                                                                                                                                                                         ']


100%|██████████| 10/10 [00:42<00:00,  4.23s/it]


In [12]:
v[0]

{'story_name': 'the-lady-of-the-moon',
 'story_section': 'in the days of the emperor yau lived a prince by the name of hou i , who was a mighty hero and a good archer . once ten suns rose together in the sky , and shone so brightly and burned so fiercely that the people on earth could not endure them . so the emperor ordered hou i to shoot at them . and hou i shot nine of them down from the sky . besides his bow , hou i also had a horse which ran so swiftly that even the wind could not catch up with it .',
 'question': 'who was a mighty hero and a good archer ?',
 'answer1': 'hou i .',
 'answer2': 'hou i .',
 'local_or_sum': 'local',
 'attribute': 'character',
 'ex_or_im': 'explicit',
 'ex_or_im2': 'explicit'}

In [49]:
val[807]

{'story_name': 'the-lady-of-the-moon',
 'story_section': 'in the days of the emperor yau lived a prince by the name of hou i , who was a mighty hero and a good archer . once ten suns rose together in the sky , and shone so brightly and burned so fiercely that the people on earth could not endure them . so the emperor ordered hou i to shoot at them . and hou i shot nine of them down from the sky . besides his bow , hou i also had a horse which ran so swiftly that even the wind could not catch up with it .',
 'question': 'who was a mighty hero and a good archer ?',
 'answer1': 'hou i .',
 'answer2': 'hou i .',
 'local_or_sum': 'local',
 'attribute': 'character',
 'ex_or_im': 'explicit',
 'ex_or_im2': 'explicit'}

In [57]:
valid[valid['answer1']=='hou i .']

,story_name,story_section,question,answer1,answer2,local_or_sum,attribute,ex_or_im,ex_or_im2
645,the-lady-of-the-moon,in the days of the emperor yau lived a prince ...,who was a mighty hero and a good archer ?,hou i .,hou i .,local,character,explicit,explicit


In [19]:
%%time
val_results_df = process_dataset(val)

# save results
results_path = 'drive/MyDrive/266/FinalProject/results/'  #DANIELLE
val_results_df.to_csv(results_path+f'T5baseline_val.csv', index=False)

 10%|▉         | 100/1025 [05:33<51:54,  3.37s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.1250


 20%|█▉        | 200/1025 [11:11<48:17,  3.51s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.0000


 29%|██▉       | 300/1025 [17:07<43:08,  3.57s/it]

...


Processed 900 samples; Results length: 900
Latest Answer1 ROUGE-1: 1.0000


 98%|█████████▊| 1000/1025 [56:38<01:32,  3.71s/it]


Processed 1000 samples; Results length: 1000
Latest Answer1 ROUGE-1: 0.0000


100%|██████████| 1025/1025 [58:13<00:00,  3.41s/it]


CPU times: user 57min 49s, sys: 14.2 s, total: 58min 3s
Wall time: 58min 14s


# LOOK AT RESULTS

In [22]:
results_path = '/content/drive/MyDrive/266/FinalProject/results'
df_v = pd.read_csv(f'{results_path}/T5baseline_val.csv')
print(f"{len(df_v)} records in validation results")
df_v[:2]

1025 records in validation results


,id,question,ex_or_im,story_section,reference_answer1,reference_answer2,generated_answer,rouge1_a1,rouge2_a1,rougeL_a1,...,translation_length_a1,reference_length_a1,bleu_a2,precisions_a2,brevity_penalty_a2,length_ratio_a2,translation_length_a2,reference_length_a2,bleurt_score_a1,bleurt_score_a2
0,0,why was madge unable to resist the boy's reque...,explicit,his tone was so pleading that mistress madge w...,his tone was so pleading .,his tone was so pleading that mistress madge w...,his tone was so pleading that the mistress mad...,0.526316,0.470588,0.526316,...,15,6,0.815355,"[0.9375, 0.8666666666666667, 0.785714285714285...",1.0,1.071429,15,14,-0.185938,0.992595
1,1,what did the sparrows do after the old man cho...,explicit,the old man could not refuse this kind proposa...,helped him put it on his back .,the sparrows all helped him put it on his back .,the sparrows helped him put it on his back and...,0.560000,0.521739,0.560000,...,19,8,0.388172,"[0.55, 0.42105263157894735, 0.3333333333333333...",1.0,1.727273,19,11,-0.518739,0.149364


In [33]:
pd.set_option('display.max_columns', None)

results_path = '/content/drive/MyDrive/266/FinalProject/results'
df_r = pd.read_csv(f'{results_path}/T5baseline_test.csv')

In [34]:
print(len(df_r))

1007


In [35]:
df_r[:5]

,id,question,ex_or_im,story_section,reference_answer1,reference_answer2,generated_answer,rouge1_a1,rouge2_a1,rougeL_a1,rougeLsum_a1,rouge1_a2,rouge2_a2,rougeL_a2,rougeLsum_a2,bleu_a1,precisions_a1,brevity_penalty_a1,length_ratio_a1,translation_length_a1,reference_length_a1,bleu_a2,precisions_a2,brevity_penalty_a2,length_ratio_a2,translation_length_a2,reference_length_a2,bleurt_score_a1,bleurt_score_a2
0,0,how did the brother feel because they had been...,explicit,"but his brother complained of being weary , an...",weary .,weary .,they were weary,0.500000,0.000000,0.500000,0.500000,0.500000,0.000000,0.500000,0.500000,0.000000,"[0.3333333333333333, 0.0, 0.0, 0.0]",1.000000,1.500000,3,2,0.000000,"[0.3333333333333333, 0.0, 0.0, 0.0]",1.000000,1.500000,3,2,-0.087235,-0.087235
1,1,how did ian feel after he reached the place wh...,implicit,' bear me to the kingdom of the king under the...,nervous .,excited .,glad,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0]",0.367879,0.500000,1,2,0.000000,"[0.0, 0.0, 0.0, 0.0]",0.367879,0.500000,1,2,-0.983741,0.131790
2,2,what did the mice do when the they saw the cat ?,explicit,""" eating is hard work in this country , "" said...",hurried into their holes .,the mice had to hurry into their holes .,They had to hurry into their holes .,0.545455,0.444444,0.545455,0.545455,0.800000,0.769231,0.800000,0.800000,0.345721,"[0.5, 0.42857142857142855, 0.3333333333333333,...",1.000000,1.600000,8,5,0.742088,"[0.875, 0.8571428571428571, 0.8333333333333334...",0.882497,0.888889,8,9,-0.031122,-0.112637
3,3,why did the prince not tell his mother the tru...,explicit,"the queen spoke several times to her son , to ...",she was of the race of the ogres .,she was the race of the ogres .,she had all the difficulty in the world to avo...,0.285714,0.000000,0.285714,0.285714,0.300000,0.000000,0.300000,0.300000,0.000000,"[0.23076923076923078, 0.0, 0.0, 0.0]",1.000000,1.444444,13,9,0.000000,"[0.23076923076923078, 0.0, 0.0, 0.0]",1.000000,1.625000,13,8,-1.316074,-1.439150
4,4,why were the women frightened ?,explicit,as the two ladies leaned over the side of the ...,never before had they seen the face of mortal ...,never before have they seen the face of a mort...,they saw the face of mortal man .,0.705882,0.533333,0.705882,0.705882,0.666667,0.375000,0.666667,0.666667,0.485987,"[0.875, 0.7142857142857143, 0.6666666666666666...",0.687289,0.727273,8,11,0.000000,"[0.875, 0.5714285714285714, 0.3333333333333333...",0.606531,0.666667,8,12,0.335033,0.225543


In [36]:
len(df_r[df_r['reference_answer1']!=df_r['reference_answer2']])

722

In [21]:
len(df_r)

5

# Analysis of Results

In [6]:
# Calculate and print average ROUGE scores
avg_scores1 = df_r[['rouge1_a1', 'rouge2_a1', 'rougeL_a1']].mean()
print("\nAverage A1 ROUGE Scores:")
for metric, score in avg_scores1.items():
   print(f"{metric}: {score:.4f}")

avg_scores2 = df_r[['rouge1_a2', 'rouge2_a2', 'rougeL_a2']].mean()
print("\nAverage A2 ROUGE Scores:")
for metric, score in avg_scores2.items():
   print(f"{metric}: {score:.4f}")



# Calculate average by question type
avg_by_type1 = df_r.groupby(['ex_or_im'])[['rouge1_a1', 'rouge2_a1', 'rougeL_a1']].mean()
print("\nAverage A1 ROUGE Scores by Question type:")
print(avg_by_type1)

avg_by_type2 = df_r.groupby(['ex_or_im'])[['rouge1_a2', 'rouge2_a2', 'rougeL_a2']].mean()
print("\nAverage A2 ROUGE Scores by Question type:")
print(avg_by_type2)

# for i in t.itertuples():
#   print(f"\nAverage {i.Index} ROUGE scores:")
#   for metric, score in zip(metrics, i[1:]):
#     print(f"{metric}: {score:.4f}")

# Print some example summaries
print("\nExample Summaries:")
for i in range(min(10, len(df_r))):
   print(f"\nExample {i}:")
   print(f"Reference1: {df_r.iloc[i]['reference_answer1']}")
   print(f"Reference2: {df_r.iloc[i]['reference_answer2']}")
   print(f"Generated: {df_r.iloc[i]['generated_answer']}")


Average A1 ROUGE Scores:
rouge1_a1: 0.2017
rouge2_a1: 0.1500
rougeL_a1: 0.2017

Average A2 ROUGE Scores:
rouge1_a2: 0.2160
rouge2_a2: 0.1014
rougeL_a2: 0.2160

Average A1 ROUGE Scores by Question type:
          rouge1_a1  rouge2_a1  rougeL_a1
ex_or_im                                 
explicit   0.319865       0.25   0.319865
implicit   0.024390       0.00   0.024390

Average A2 ROUGE Scores by Question type:
          rouge1_a2  rouge2_a2  rougeL_a2
ex_or_im                                 
explicit   0.271132   0.117647   0.271132
implicit   0.133333   0.076923   0.133333

Example Summaries:

Example 0:
Reference1: sat together , rejoicing in their new - found happiness .
Reference2: for a little time they sat together .
Generated: shook her head

Example 1:
Reference1: alarmed .
Reference2: alarmed .
Generated: the doctor was alarmed at rin jin 's evident displeasure .

Example 2:
Reference1: the young men did not see ahti with their own eyes .
Reference2: it does n't sound real .


In [7]:

pd.set_option('display.max_columns', None)